In [1]:
from teradataml import *
import chess
import time
import random


In [2]:
# Connect with Teradata Database
eng = create_context("192.168.56.1", "val", "val")
conn = get_connection()

# Drop the tables if they already exist - if you want to start over with the disctionary.

#if conn.dialect.has_table(connection=conn, table_name="replay_buffer"):
#    db_drop_table("replay_buffer")

# Create the tables holding the training data.

#create_game_table = "create multiset table replay_buffer (game_id BIGINT, sequence INTEGER, FEN VARCHAR(100), ply VARCHAR(10), reward FLOAT) PRIMARY INDEX (game_id);"
#execute_sql(create_game_table)

#execute_sql("insert into replay_buffer (game_id) values (0);")

In [3]:
def play_game(in_board, training=None):
    
    # Initialize the starting variables.
    sequence_no = 0
    reward = 0


    board = chess.Board()
    if in_board is not None:
        board = in_board

    if training == True:
        
        # Set the game_id by looking up the last used game_id.
        game_id_cursor = execute_sql("SELECT MAX(game_id) FROM replay_buffer;")
        game_id = int(game_id_cursor.fetchall()[0][0]) + 1  # There has to be a better way to do this.

    # Launch into the key loop.
    while not board.is_checkmate() and not board.is_stalemate() and not board.is_insufficient_material() and not board.can_claim_draw() and not board.is_seventyfive_moves() and not board.is_fivefold_repetition():
        if training and sequence_no == 0:
            fen = board.fen()
            spaces_pos = [pos for pos, char in enumerate(fen) if char == ' ']
            opening_fen = fen[:spaces_pos[3]]
            
            insert_replay_buffer = "INSERT INTO replay_buffer (game_id, ply, FEN) VALUES (" + str(game_id) + "," + str(sequence_no) + ",'" + opening_fen + "');"
            execute_sql(insert_replay_buffer)

        sequence_no += 1
        all_moves = str(board.legal_moves)
        start_pos = [pos for pos, char in enumerate(all_moves) if char == '(']
        end_pos = [pos for pos, char in enumerate(all_moves) if char == ')']
        all_moves_string= all_moves[start_pos[0]+1:end_pos[0]]
        all_moves_string = all_moves_string.replace(" ", "")
        all_moves_array = all_moves_string.split(',')
        next_move = random.choice(all_moves_array)
        board.push_san(next_move)

        if training:
            fen = board.fen()
            spaces_pos = [pos for pos, char in enumerate(fen) if char == ' ']
            trimmed_fen = fen[:spaces_pos[3]]
            
            insert_replay_buffer = "insert into replay_buffer (game_id, ply, FEN, move) values (" + str(game_id) + "," + str(sequence_no) + ",'" + trimmed_fen + "','" + next_move + "');"
            execute_sql(insert_replay_buffer)
            if board.is_checkmate():
                if board.outcome().winner:
                    reward = 1
                else:
                    reward = -1

    print("The reward for " + str(game_id) + " is:" + str(reward) + ".")
    execute_sql("UPDATE replay_buffer SET reward = " + str(reward) + " WHERE game_id = " + str(game_id) + ";")

        


    return board

In [4]:
num_games = 1374

for counter in range(num_games):
    start_time = time.time()
    play_game(None,training=True)
    end_time = time.time()
    print(str(counter+1) + " of " + str(num_games) + " completed. Last game duration was " + str(end_time-start_time) + ".")

The reward for 8627 is:0.
1 of 1374 completed. Last game duration was 8.345584869384766.
The reward for 8628 is:0.
2 of 1374 completed. Last game duration was 8.108362436294556.
The reward for 8629 is:0.
3 of 1374 completed. Last game duration was 5.2567524909973145.
The reward for 8630 is:0.
4 of 1374 completed. Last game duration was 10.356947898864746.
The reward for 8631 is:0.
5 of 1374 completed. Last game duration was 8.42882251739502.
The reward for 8632 is:0.
6 of 1374 completed. Last game duration was 8.324636459350586.
The reward for 8633 is:0.
7 of 1374 completed. Last game duration was 10.78262734413147.
The reward for 8634 is:0.
8 of 1374 completed. Last game duration was 8.931396722793579.
The reward for 8635 is:0.
9 of 1374 completed. Last game duration was 6.022091627120972.
The reward for 8636 is:0.
10 of 1374 completed. Last game duration was 6.266726016998291.
The reward for 8637 is:0.
11 of 1374 completed. Last game duration was 13.25977110862732.
The reward for 863

In [5]:
conn.close

<bound method Connection.close of <sqlalchemy.engine.base.Connection object at 0x00000205DCF3DCD0>>

In [6]:
remove_context()

True